In [1]:
import keras
from keras.utils.np_utils import to_categorical

import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('./DATA/sensorless_data.csv')
data.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F40,F41,F42,F43,F44,F45,F46,F47,F48,TARGET
0,-3.014600e-07,8.260300e-06,-0.000012,-0.000002,-1.438600e-06,-0.000021,0.031718,0.031710,0.031721,-0.032963,...,-0.63308,2.9646,8.1198,-1.4961,-1.4961,-1.4961,-1.4996,-1.4996,-1.4996,1
1,2.913200e-06,-5.247700e-06,0.000003,-0.000006,2.778900e-06,-0.000004,0.030804,0.030810,0.030806,-0.033520,...,-0.59314,7.6252,6.1690,-1.4967,-1.4967,-1.4967,-1.5005,-1.5005,-1.5005,1
2,-2.951700e-06,-3.184000e-06,-0.000016,-0.000001,-1.575300e-06,0.000017,0.032877,0.032880,0.032896,-0.029834,...,-0.63252,2.7784,5.3017,-1.4983,-1.4983,-1.4982,-1.4985,-1.4985,-1.4985,1
3,-1.322600e-06,8.820100e-06,-0.000016,-0.000005,-7.282900e-07,0.000004,0.029410,0.029401,0.029417,-0.030156,...,-0.62289,6.5534,6.2606,-1.4963,-1.4963,-1.4963,-1.4975,-1.4975,-1.4976,1
4,-6.836600e-08,5.666300e-07,-0.000026,-0.000006,-7.940600e-07,0.000013,0.030119,0.030119,0.030145,-0.031393,...,-0.63010,4.5155,9.5231,-1.4958,-1.4958,-1.4958,-1.4959,-1.4959,-1.4959,1


### Format input data

In [3]:
x_cols = list(data.columns[:-1])
X_data = data[x_cols].copy()
# Adding 0 for easy reshaping
X_data['F49'] = 0
X_data.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F40,F41,F42,F43,F44,F45,F46,F47,F48,F49
0,-3.014600e-07,8.260300e-06,-0.000012,-0.000002,-1.438600e-06,-0.000021,0.031718,0.031710,0.031721,-0.032963,...,-0.63308,2.9646,8.1198,-1.4961,-1.4961,-1.4961,-1.4996,-1.4996,-1.4996,0
1,2.913200e-06,-5.247700e-06,0.000003,-0.000006,2.778900e-06,-0.000004,0.030804,0.030810,0.030806,-0.033520,...,-0.59314,7.6252,6.1690,-1.4967,-1.4967,-1.4967,-1.5005,-1.5005,-1.5005,0
2,-2.951700e-06,-3.184000e-06,-0.000016,-0.000001,-1.575300e-06,0.000017,0.032877,0.032880,0.032896,-0.029834,...,-0.63252,2.7784,5.3017,-1.4983,-1.4983,-1.4982,-1.4985,-1.4985,-1.4985,0
3,-1.322600e-06,8.820100e-06,-0.000016,-0.000005,-7.282900e-07,0.000004,0.029410,0.029401,0.029417,-0.030156,...,-0.62289,6.5534,6.2606,-1.4963,-1.4963,-1.4963,-1.4975,-1.4975,-1.4976,0
4,-6.836600e-08,5.666300e-07,-0.000026,-0.000006,-7.940600e-07,0.000013,0.030119,0.030119,0.030145,-0.031393,...,-0.63010,4.5155,9.5231,-1.4958,-1.4958,-1.4958,-1.4959,-1.4959,-1.4959,0


In [4]:
len(X_data)

58509

In [5]:
X = X_data.to_numpy().reshape((len(X_data),7,7,1))
X.shape

(58509, 7, 7, 1)

In [6]:
X[0].shape

(7, 7, 1)

In [7]:
data['TARGET'] = data['TARGET'] - 1
y = data['TARGET'].to_numpy()

In [8]:
y = to_categorical(y, num_classes=data['TARGET'].nunique())

### Randomizing dataset

In [9]:
idxs = np.arange(len(X))
samples = np.random.choice(idxs,size=10000)

X_rand = X[samples]
y_rand = y[samples]

### CNN Model

In [10]:
model = keras.models.Sequential([
    keras.layers.Conv2D(256, (2,2), strides=1, activation='relu', input_shape=(7,7,1)),
    keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    keras.layers.Conv2D(256, (2,2), strides=1, activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(11, activation='softmax')
])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 6, 6, 256)         1280      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 3, 3, 256)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 2, 2, 256)         262400    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 1, 256)         0         
_________________________________________________________________
flatten (Flatten)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 11)                5

In [12]:
y_rand[:25000][0].shape

(11,)

In [13]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
history = model.fit(X_rand , y_rand, steps_per_epoch=100, epochs = 10)

Epoch 1/10
100/100 [==============================] - 1s 12ms/step - loss: 2.4134 - accuracy: 0.1182
Epoch 2/10
100/100 [==============================] - 1s 13ms/step - loss: 1.9240 - accuracy: 0.3166
Epoch 3/10
100/100 [==============================] - 2s 18ms/step - loss: 1.3196 - accuracy: 0.4596
Epoch 4/10
100/100 [==============================] - 1s 13ms/step - loss: 1.1405 - accuracy: 0.5480
Epoch 5/10
100/100 [==============================] - 1s 12ms/step - loss: 0.9524 - accuracy: 0.6297
Epoch 6/10
100/100 [==============================] - 1s 12ms/step - loss: 0.8070 - accuracy: 0.6671
Epoch 7/10
100/100 [==============================] - 1s 12ms/step - loss: 0.7174 - accuracy: 0.6994
Epoch 8/10
100/100 [==============================] - 1s 12ms/step - loss: 0.6648 - accuracy: 0.7236
Epoch 9/10
100/100 [==============================] - 1s 12ms/step - loss: 0.6153 - accuracy: 0.7430
Epoch 10/10
100/100 [==============================] - 1s 13ms/step - loss: 0.5529 - accura

In [38]:
history.history['accuracy'][-1]

0.7766000032424927

### Running Experiment

In [29]:
all_fold_idxs = np.random.choice(idxs, size=len(idxs))

nfolds = 3
fold_size = int(len(all_fold_idxs)/3)
folds = [all_fold_idxs[i:i+fold_size] for i in range(0, len(all_fold_idxs), fold_size)]

In [45]:
X_rand.shape

(10000, 7, 7, 1)

In [44]:
X_train.shape

(2, 7, 7, 1)

In [48]:
np.concatenate([[1,2],[3,4]])

array([1, 2, 3, 4])

In [51]:
X_test

array([55450, 10543,  8311, ..., 34351, 30668, 49312])

In [62]:
X_rand[0].shape

(7, 7, 1)

In [65]:
X_test[0].shape

(7, 7, 1)

In [81]:
train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []
for fold_idx in range(nfolds):
    print("Running on fold " + str(fold_idx))
    X_test = X[folds[fold_idx]]
    y_test = y[folds[fold_idx]]
    
    train_fold_idxs = [i for i in range(nfolds) if i != fold_idx]
    train_idxs = np.concatenate([list(folds[idx]) for idx in train_fold_idxs])
    X_train = X[train_idxs]
    y_train = y[train_idxs]
    
    model = keras.models.Sequential([
        keras.layers.Conv2D(256, (2,2), strides=1, activation='relu', input_shape=(7,7,1)),
        keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
        keras.layers.Conv2D(256, (2,2), strides=1, activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
        keras.layers.Flatten(),
        keras.layers.Dense(512, activation='relu'),
        keras.layers.Dense(11, activation='softmax')
    ])
    
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])
    history = model.fit(X_train , y_train, steps_per_epoch=100, epochs = 10)
    
    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['categorical_accuracy'][-1]

    test_preds = model.predict(X_test)

    loss_calc = keras.metrics.CategoricalCrossentropy()
    loss_calc.reset_states()
    loss_calc.update_state(y_test, test_preds)
    test_loss = loss_calc.result().numpy()

    acc_calc = keras.metrics.CategoricalAccuracy()
    acc_calc.reset_states()
    acc_calc.update_state(y_test, test_preds)
    test_accuracy = acc_calc.result().numpy()

    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)

Running on fold 0
Epoch 1/10
100/100 [==============================] - 6s 58ms/step - loss: 2.3048 - categorical_accuracy: 0.1712
Epoch 2/10
100/100 [==============================] - 5s 53ms/step - loss: 1.3785 - categorical_accuracy: 0.4659
Epoch 3/10
100/100 [==============================] - 6s 57ms/step - loss: 1.0902 - categorical_accuracy: 0.5615
Epoch 4/10
100/100 [==============================] - 5s 51ms/step - loss: 0.8931 - categorical_accuracy: 0.6504
Epoch 5/10
100/100 [==============================] - 6s 57ms/step - loss: 0.7352 - categorical_accuracy: 0.7087
Epoch 6/10
100/100 [==============================] - 5s 55ms/step - loss: 0.6690 - categorical_accuracy: 0.7381
Epoch 7/10
100/100 [==============================] - 5s 55ms/step - loss: 0.5765 - categorical_accuracy: 0.7674
Epoch 8/10
100/100 [==============================] - 5s 55ms/step - loss: 0.5108 - categorical_accuracy: 0.7901
Epoch 9/10
100/100 [==============================] - 6s 57ms/step - loss: 0.4

In [82]:
np.mean(test_accuracies)

0.82182235